In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("DataIngestionPipeline") \
    .getOrCreate()

In [77]:
import subprocess
import requests
from bs4 import BeautifulSoup
import re
import os
from urllib.parse import urljoin


In [58]:
data_url = "https://cricsheet.org/downloads/"

In [59]:
response = requests.get(data_url)

In [60]:
display(response.text)

'<!DOCTYPE html>\n\n\n\n <html lang="en-gb" class="no-js"> \n<head>\n  <meta http-equiv="content-type" content="text/html; charset=utf-8">\n<meta name="mobile-web-app-capable" content="yes" />\n<meta name="viewport" content="width=device-width, initial-scale=1.0">\n<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n<link rel="stylesheet" href="/css/main.css">\n<link rel="alternate" type="application/rss+xml" href=\'http://cricsheet.org/articles.xml\' />\n\n<script src="/js/vendor/modernizr-2.6.1.min.js"></script>\n<script type="text/javascript" src="//use.typekit.net/mjy1jks.js"></script>\n<script type="text/javascript">try{Typekit.load();}catch(e){}</script>\n<title>Available match data downloads - Cricsheet</title>\n\n</head>\n\n<body>\n  <div class="wrapper wrapper_banner">\n  <div id="banner">\n    <header>\n      <h1><a href="/">Cricsheet</a></h1>\n      <h2>Freely-available structured data for cricket, including ball-by-ball data international and T20 League cricket 

In [87]:
if response.status_code == 200:
    # Parse the content with BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    pattern = r'/downloads/[a-z0-9A-Z]+'

    # Find all the <a> tags within <li> tags
    ul_elements = soup.find_all('ul', class_='formatlinks')  # You can adjust this if the <li> tags are nested in specific classes or IDs
    filtered_links = []

    for ul in ul_elements:
        a_tags = ul.find_all('a', href=True)

        for a_tag in a_tags:
            href = a_tag['href']
            if "male" not in href and "json" in href:
                filtered_links.append(href)

    td_elements = soup.find_all('td', class_='name', attrs={'rowspan': False})

    for td in td_elements:
        # Find the <span> with class 'withheld' and decompose it (remove it)
        withheld_span = td.find('span', class_='withheld')
        if withheld_span:
            withheld_span.decompose()


    print(f"Number of filtered links: {len(filtered_links)}")
    print(f"Number of <td> elements: {len(td_elements)}")
    print(f"--------------------------------")

    notebook_dir = os.getcwd()
    parent_dir = os.path.dirname(notebook_dir)
    data_dir = os.path.join(parent_dir, 'data')

    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
        
    links_file_path = os.path.join(data_dir, 'links.csv')
    
    data_url2 = "https://cricsheet.org/"

    filtered_links = [urljoin(data_url2, link) for link in filtered_links]
    td_elements = [td.text.strip() for td in td_elements]
    
    data = list(zip(td_elements, filtered_links))

    columns = ["Event/Country Name", "Links"]
    df = spark.createDataFrame(data, columns)

    df.write.csv(links_file_path, header=True, mode='overwrite')


else:
    print(f"Failed to retrieve the webpage. Status code: {response.status_code}")

Number of filtered links: 422
Number of <td> elements: 422
--------------------------------
